In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from muxcnn.utils import load_params, load_img
import torch
from muxcnn.resnet_muxconv import ResNet_MuxConv

## Torch model

In [3]:
from muxcnn.models.ResNet20 import ResNet, BasicBlock

model = ResNet(BasicBlock,[1,1,1])
model.eval() ########## 필수! 
load_params(model, fn_param="./ResNet8.pt",device='cpu')

In [4]:
img_tensor = load_img("./cute.jpg", hi=32, wi=32)

In [5]:
print(model(img_tensor))

before softmax tensor([[ -9.3005, -11.0586,  -2.9741,  -2.9179,  -9.3500,   4.1474,  -6.8721,
          -3.6999,  -8.9446,  -7.6901]], grad_fn=<AddmmBackward0>)
tensor([[-1.3450e+01, -1.5208e+01, -7.1236e+00, -7.0674e+00, -1.3500e+01,
         -2.0791e-03, -1.1022e+01, -7.8494e+00, -1.3094e+01, -1.1840e+01]],
       grad_fn=<LogSoftmaxBackward0>)


# MuxCNN

In [6]:
muxed_model = ResNet_MuxConv(model, alpha=12)

functions set
degrees = [15, 15, 15, 15], margin = 0.01, eps = 0.02


In [7]:
result = fhe_model.forward_early(img_tensor)

NameError: name 'time' is not defined

# FHE Context

In [7]:
import numpy as np
import pandas as pd

from hemul.cipher import *
from hemul.ciphertext import Plaintext
from hemul.scheme import *

from hemul.algorithms import Algorithms 
from hemul.stats import Statistics
from hemul.context import set_all

from math import sqrt

In [8]:
def gen_new_ctxt(ev):
    this_contxt = ev.context.params
    newctxt = CiphertextStat(logp = this_contxt.logp,
                          logq = this_contxt.logq,
                          logn = this_contxt.logn)
    newctxt._enckey_hash = key_hash(ev.context.enc_key)
    return newctxt

In [9]:
context, ev, encoder, encryptor, decryptor = set_all(30, 600, 15)
nslots = context.params.nslots

In [10]:
from muxcnn.utils import get_channel_last, get_conv_params
from muxcnn.hecnn_par import MultParPack, parMuxBN, ParMultWgt, \
    tensor_multiplexed_shifted_weight_par, tensor_multiplexed_selecting, Vec
imgl = get_channel_last(img_tensor[0].detach().numpy())
ki = 1 # initial ki
hi, wi, ch = imgl.shape

# early conv and bn
_, ins0, outs0 = get_conv_params(model.conv1, {'k':ki, 'h':hi, 'w':wi})
ct_a = MultParPack(imgl, ins0)

In [51]:
def SumSlots(ct_a,m,p):
    """Addition only"""
    nrots = 0
    n = int(np.floor(np.log2(m)))
    #ct_b = []
    #ct_b.append(ct_a._arr)
    ctx_b = [] ####
    ctx_b.append(ct_a) ####
    for j in range(1,n+1):
        lrots = int(p*2**(j-1))
        #ct_b.append(ct_b[j-1]+np.roll(ct_b[j-1],-lrots))
        ctx_b.append(ev.add(ctx_b[j-1], 
                            ev.lrot(ctx_b[j-1], lrots, inplace=False),
                           inplace=False)) ####
        if lrots!=0:
            nrots=nrots+1  #______________________________ROTATION
    #ct_c = ct_b[n]
    ctx_c = ctx_b[n] ####
    for j in range(0,n):
        n1 = np.floor((m/(2**j))%2)
        if n1==1:
            n2 =int(np.floor((m/(2**(j+1)))%2))
            lrots = int(p*2**(j+1))*n2
            #ct_c += np.roll(ct_b[j],-lrots)
            ev.add(ctx_c, 
                   ev.lrot(ctx_b[j],lrots, inplace=False),
                  inplace=True) ####
            if lrots!=0:
                nrots=nrots+1#____________________________ROTATION
    #print(ctx_c._arr == ct_c) ####
    return ctx_c,nrots

In [85]:
from muxcnn.utils import get_p, get_q, get_t 
def MultParConvBN_fhe(ct_a, U, bn_layer, ins:Dict, outs:Dict,
                    kernels=[3,3],
                    nslots=2**15, 
                    scale_factor=1):
    hi,wi,ci,ki,ti,pi = [ins[k] for k in ins.keys()]
    ho,wo,co,ko,to,po = [outs[k] for k in outs.keys()]
    q = get_q(co,pi)
    fh,fw= kernels[0],kernels[1]
    print(f"[MultParConv] (hi,wi,ci,ki,ti,pi) =({hi:2},{wi:2},{ci:2},{ki:2},{ti:2}, {pi:2})")
    print(f"[MultParConv] (ho,wo,co,ko,to,po) =({ho:2},{wo:2},{co:2},{ko:2},{to:2}, {po:2})")
    print(f"[MultParConv] q = {q}")

    MuxBN_C, MuxBN_M, MuxBN_I = parMuxBN(bn_layer, outs, nslots)

    #ct_d = np.zeros(nslots)
    ct_d = gen_new_ctxt(ev) ####
    ev.mod_down_by(ct_d, 2*ct_d.logp)
    ct = []
    nrots=0
    for i1 in range(fh):
        temp = []
        for i2 in range(fw):
            lrots = int((-(ki**2)*wi*(i1-(fh-1)/2) - ki*(i2-(fw-1)/2))) #both neg in the paper, git -,+
            #temp.append(np.roll(ct_a,lrots))
            temp.append(ev.lrot(ct_a, -lrots, inplace=False))
            if lrots!=0:
                nrots = nrots+ 1#____________________________________ROTATION
        ct.append(temp)

    for i3 in range(q):
        #ct_b = np.zeros(nslots)
        ct_b = gen_new_ctxt(ev) ####
        ev.mod_down_by(ct_b, ct_b.logp)
        for i1 in range(fh):
            for i2 in range(fw):
                w = ParMultWgt(U,i1,i2,i3,ins,co,kernels,nslots)
                #ct_b += ct[i1][i2]*w                
                w_enc = encoder.encode(w, ct[i1][i2].logp) ####
                tmp = ev.mult_by_plain(ct[i1][i2], w_enc, inplace=False)
                ev.rescale_next(tmp)
                ev.add(ct_b, tmp, inplace=True) ####
        
        #ev.rescale_next(ct_b)
        #print("ct_b", ct_b._arr)#[2000:2010])
        ct_c,nrots0 = SumSlots(ct_b, ki,              1)
        ct_c,nrots1 = SumSlots(ct_c, ki,          ki*wi)
        ct_c,nrots2 = SumSlots(ct_c, ti,  (ki**2)*hi*wi)
        #print("ct_c",ct_c._arr)#[2000:2010])
        nrots += nrots0 + nrots1 + nrots2#____________________________________ROTATION

        
        for i4 in range(0,min(pi,co-pi*i3)):
            i = pi*i3 +i4
            r0 = int(np.floor(nslots/pi))*(i%pi)
            r1 = int(np.floor(i/(ko**2)))*ko**2*ho*wo
            r2 = int(np.floor((i%(ko**2))/ko))*ko*wo
            r3 = i%ko
            rrots = (-r1-r2-r3)+r0
            #rolled =  np.roll(ct_c, -rrots)
            rolled = ev.lrot(ct_c, rrots, inplace=False)
            S_mp = tensor_multiplexed_selecting(ho,wo,co,ko,to,i)
            vec_S = Vec(S_mp,nslots)
            #ct_d += rolled*vec_S * MuxBN_C
            tmp = ev.mult_by_plain(rolled, 
                                   encoder.encode(vec_S * MuxBN_C, 
                                                  rolled.logp), 
                                   inplace=False)
            ev.rescale_next(tmp)
            ev.add(ct_d, tmp, inplace=True)
            if rrots!=0:
                nrots=nrots+1 #_________________________________________ROTATION

    #ev.rescale_next(ct_d)
    #print("ct_d", ct_d._arr[2040:2050])
    for j in range(int(np.round(np.log2(po)))):
        r = -int(np.round(2**j*(nslots/po)))
        #ct_d += np.roll(ct_d,-r)
        ev.add(ct_d, ev.lrot(ct_d, r, inplace=False), inplace=True)
        if r !=0:
            nrots+=1
            #ic(po,j)
    
    #print("ct_d", ct_d._arr[2040:2050])
    #ct_d -= 1/scale_factor*(MuxBN_C*MuxBN_M-MuxBN_I)
    plain_vec = encoder.encode(-1/scale_factor*(MuxBN_C*MuxBN_M-MuxBN_I), 
                              ct_d.logp)
    #print("last", ct_d.logp, ct_d.logq, plain_vec.logp)
    ev.add_plain(ct_d, plain_vec, inplace=True)

    return ct_d

In [73]:
def forward_convbn_par_fhe(cnn_layer, bn_layer, ctx, ins, kernels=[3,3]):
    U, ins, outs = get_conv_params(cnn_layer, ins)
    out = MultParConvBN_fhe(ctx, U, bn_layer, ins, outs, kernels)
    return out

In [11]:
# FHE
ctx_a = encryptor.encrypt(ct_a)
ctxt = forward_convbn_par_fhe(model.conv1, 
                          model.bn1, ctx_a, ins0)

In [87]:
# Muxed
U, ins1, outs1 = get_conv_params(model.conv1, {'k':ki, 'h':hi, 'w':wi})
ct_a = MultParPack(imgl, ins1)
from muxcnn.hecnn_par import forward_convbn_par
out1, un1 = forward_convbn_par(model.conv1, 
                              model.bn1, ct_a, ins1)

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32, 3, 1, 3,  8)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 2
2.5291 s


In [88]:
print(ctxt._arr[2000:2050] == out1[2000:2050])

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True]


## Activation

In [ ]:
if do_activation:
    out1 = appr_relu(out1, xmin=-10, xmax=10, alpha=12)

In [20]:
t0 = time()
result = fhe_model(img_tensor)
print(f"{time() - t0:.4f} s")

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32, 3, 1, 3,  8)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 2
1.9421 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 8
7.5915 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 8
7.5881 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(16,16,32, 2, 8,  4)
[MultParConv] q = 16
14.5817 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(16,16,32, 2, 8,  4)
[MultParConv] (ho,wo,co,ko,to,po) =(16,16,32, 2, 8,  4)
[MultParConv] q = 8
6.9168 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(16,16,32, 2, 8,  4)
[MultParConv] q = 16
2.1417 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(16,16,32, 2, 8,  4)
[MultParConv] (ho,wo,co,ko,to,po) =( 8, 8,64, 4, 4,  8)
[MultParConv] q = 16
13.3325 s
[MultPar

In [21]:
torch.tensor(result[::64][:10])

tensor([ -9.2678, -10.8150,  -3.1036,  -2.9026,  -9.4616,   4.3280,  -6.9752,
         -3.5619,  -8.8842,  -7.5630], dtype=torch.float64)

In [10]:
torch.tensor(result[::64][:10])

tensor([ -9.2678, -10.8150,  -3.1036,  -2.9026,  -9.4616,   4.3280,  -6.9752,
         -3.5619,  -8.8842,  -7.5630], dtype=torch.float64)